In [1]:
import tensorflow as tf
import os

In [2]:
#Avoid OOm errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Removing dodgy images

In [3]:
import cv2
import imghdr

In [4]:
data_dir = 'data'
image_exts = ['jpeg','jpg','bmp','png']

In [5]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("issue with image {}".format((image_path)))

# Loading the dataset

In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 157 files belonging to 2 classes.


In [8]:
data_iterator = data.as_numpy_iterator()

In [9]:
batch = data_iterator.next()

In [10]:
#Images represented as numpy arrays
batch[0].shape

(8, 256, 256, 3)

In [11]:
#Image labels
batch[1]

array([0, 1, 0, 0, 1, 1, 0, 1])

# 2. Preprocesing the data

## 2.1 Scale Data

In [12]:
data = data.map(lambda x,y :(x/255,y))

In [13]:
scaled_iterator = data.as_numpy_iterator()

In [14]:
batch = scaled_iterator.next()

## 2.2 Split the data

In [16]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [20]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size).skip(val_size).take(test_size)

# 3. Deep Learning Model

## 3.1 Building the deep learning model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [24]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 16)      

## 3.2 Training the model

In [28]:
logdir='logs'

In [29]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [30]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
14/14 [==============================] - 9s 420ms/step - loss: 0.9124 - accuracy: 0.5536 - val_loss: 0.6135 - val_accuracy: 0.5938
Epoch 2/20
14/14 [==============================] - 6s 407ms/step - loss: 0.6397 - accuracy: 0.6518 - val_loss: 0.6047 - val_accuracy: 0.6250
Epoch 3/20
14/14 [==============================] - 8s 545ms/step - loss: 0.6033 - accuracy: 0.7500 - val_loss: 0.5876 - val_accuracy: 0.7188
Epoch 4/20
14/14 [==============================] - 8s 527ms/step - loss: 0.5482 - accuracy: 0.7411 - val_loss: 0.4521 - val_accuracy: 0.8750
Epoch 5/20
14/14 [==============================] - 6s 424ms/step - loss: 0.3340 - accuracy: 0.9196 - val_loss: 0.5451 - val_accuracy: 0.8125
Epoch 6/20
14/14 [==============================] - 7s 477ms/step - loss: 0.3257 - accuracy: 0.8750 - val_loss: 0.3216 - val_accuracy: 0.8438
Epoch 7/20
14/14 [==============================] - 8s 569ms/step - loss: 0.2338 - accuracy: 0.9196 - val_loss: 0.1632 - val_accuracy: 0.9375
Epoch 

## 3.3 Performance Plot

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()